In [32]:
import pandas as pd
import requests, zipfile, StringIO
from BeautifulSoup import BeautifulSoup
% matplotlib inline

# Fetch Latest Seattle Counter Data

In [126]:
endpoint_dict = {'nw_58th_st_greenway_st_22nd_ave_nw': 'https://data.seattle.gov/resource/kng8-tayh.json',
                 'bgt_north_of_ne_70th': 'https://data.seattle.gov/resource/9nka-b3jn.json',
                 'eilliott_bay_trail_in_myrtle_edwards_park': 'https://data.seattle.gov/resource/65r9-nekm.json',
                 'chief_sealth_trl_north_of_thistle': 'https://data.seattle.gov/resource/t4f9-hyjz.json',
                  'mts_trail_west_of_i_90_bridge': 'https://data.seattle.gov/resource/ekqi-b8f3.json',
                 '_39th_ave_ne_greenway_at_ne_62nd_st': 'https://data.seattle.gov/resource/5cig-k5cs.json',
                 'broadway_cycle_track_north_of_e_union_st': 'https://data.seattle.gov/resource/keqs-cqp7.json',
                 '_26th_ave_sw_greenway_at_sw_oregon_st': 'https://data.seattle.gov/resource/ibyv-67da.json',
                 '_2nd_ave_cycletrack': 'https://data.seattle.gov/resource/aq6e-mcz8.json',
                 'spokane_st_bridge': 'https://data.seattle.gov/resource/v6y4-2gyc.json'}

In [128]:
df_dict = {}
for name, endpoint in endpoint_dict.iteritems():
    df_dict[name] = pd.read_json(endpoint,orient='columns')

In [141]:
# Data formats are inconsistent so we have to specify the bike columns to get totals

# Bike total column(s)
totals_dict   = {'nw_58th_st_greenway_st_22nd_ave_nw': ['east','west'],
                 'bgt_north_of_ne_70th': ['bike_north','bike_south'],
                 'eilliott_bay_trail_in_myrtle_edwards_park': ['bike_north','bike_south'],
                 'chief_sealth_trl_north_of_thistle': ['bike_north','bike_south'],
                  'mts_trail_west_of_i_90_bridge': ['bike_north','bike_south'],
                 '_39th_ave_ne_greenway_at_ne_62nd_st': ['north','south'],
                 'broadway_cycle_track_north_of_e_union_st': ['nb','sb'],
                 '_26th_ave_sw_greenway_at_sw_oregon_st': ['north','south'],
                 '_2nd_ave_cycletrack': ['nb','sb'],
                 'spokane_st_bridge': ['east','west']}

In [223]:
# Results for october
oct_counts = {}
april_counts = {}

for name, tot_cols in totals_dict.iteritems():
    df = df_dict[name]
    
    # Create new total_bike column
    df['total_bike'] = df[tot_cols].sum(axis=1)
    
    df['date_object'] = pd.to_datetime(df['date'])
    # extract separate columns for year, month, day, and hour
    df['year'] = df['date_object'].apply(lambda row: row.year)
    df['month'] = df['date_object'].apply(lambda row: row.month)
    df['day'] = df['date_object'].apply(lambda row: row.day)
    df['hour'] = df['date_object'].apply(lambda row: row.hour)
    df['weekday'] = df['date_object'].apply(lambda row: row.weekday())   # note weekends are 5 and 6, (sat and sunday)
    # Weekday indicator
    
    # Get the mean for the month of October
    oct_counts[name] = df[(df['month'] == 10) & (df['year'] == 2014) & (df['weekday'] < 5)].groupby('day')['total_bike'].sum().mean()
    april_counts[name] = df[(df['month'] == 4) & (df['year'] == 2014) & (df['weekday'] < 5)].groupby('day')['total_bike'].sum().mean()

In [225]:
df = pd.DataFrame([oct_counts.keys(),oct_counts.values()]).T
df.columns = ['location','total_bikes_october']

In [226]:
df

,location,total_bikes_october
0,broadway_cycle_track_north_of_e_union_st,411.8696
1,mts_trail_west_of_i_90_bridge,496.5652
2,_26th_ave_sw_greenway_at_sw_oregon_st,925.3913
3,_39th_ave_ne_greenway_at_ne_62nd_st,320.7826
4,_2nd_ave_cycletrack,NaN
5,chief_sealth_trl_north_of_thistle,19.3913
6,nw_58th_st_greenway_st_22nd_ave_nw,688.1304
7,bgt_north_of_ne_70th,780.3478
8,spokane_st_bridge,834.087
9,eilliott_bay_trail_in_myrtle_edwards_park,1137.696


## Connect these count locations to the 2-hour count locations


In [229]:
# load count observed data
wsdot_counts = pd.read_csv('../data/wsdot_bike_counts.txt')